In [119]:
import pandas as pd
import sqlite3

In [69]:
genstudio = pd.read_csv('genstudio.csv', index_col=0, low_memory=False)
metadata = pd.read_csv('metadata.csv', index_col=0)

In [70]:
connection = sqlite3.connect('dna.db')

In [71]:
genstudio.head()

,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022


In [72]:
metadata.head()

,dna_chip_id,breed,sex
0,202290551164R09C01,Д,Хр
1,202341831114R02C01,Д,Хр
2,202341831114R03C01,Д,Хр
3,202341831114R04C01,Д,Хр
4,202290551140R01C01,Д,Хр


In [73]:
# Position (dtype == object) --> Position (dtype == int)

genstudio = genstudio[genstudio['Position'].str.contains('-') == False]
genstudio['Position'] = pd.to_numeric(genstudio['Position'])

In [75]:
query1 = '''CREATE TABLE IF NOT EXISTS dna_snps(
                                id INTEGER PRIMARY KEY,
                                dna_id TEXT,
                                SNP TEXT,
                                SNP_name TEXT,
                                Position INTEGER,
                                GC_score INTEGER
                                )'''

query2 = '''CREATE TABLE IF NOT EXISTS dna_char(
                                id INTEGER PRIMARY KEY,
                                dna_id TEXT,
                                breed TEXT,
                                sex TEXT
                                )'''

connection.execute(query1)
connection.execute(query2)

In [76]:
insertion_query1 = f'''INSERT INTO 
                      dna_snps(dna_id, SNP, SNP_name, Position, GC_score)
                      VALUES (?, ?, ?, ?, ?)'''

insertion_query2 = f'''INSERT INTO 
                      dna_char(dna_id, breed, sex)
                      VALUES (?, ?, ?)'''

In [83]:
info1 = genstudio[['Sample ID', 'SNP', 'SNP Name', 'Position', 'GC Score']].values.tolist()

connection.executemany(insertion_query1, info1)

In [84]:
info2 = metadata[['dna_chip_id', 'breed', 'sex']].values.tolist()

connection.executemany(insertion_query2, info2)

In [85]:
connection.commit()

In [86]:
select_query = '''
                SELECT dna_id, SNP
                FROM dna_snps'''

result = connection.execute(select_query).fetchall()

In [87]:
for dna in result[:10]:
    print(*dna)

202341831114R01C01 [T/C]
202341831114R01C01 [T/C]
202341831114R01C01 [A/G]
202341831114R01C01 [A/G]
202341831114R01C01 [A/G]
202341831114R01C01 [T/C]
202341831114R01C01 [T/C]
202341831114R01C01 [A/G]
202341831114R01C01 [T/C]
202341831114R01C01 [A/G]


In [89]:
select_query = '''
                SELECT dna_id, sex
                FROM dna_char'''

result = connection.execute(select_query).fetchall()

In [90]:
for dna in result[:10]:
    print(*dna)

202290551164R09C01 Хр
202341831114R02C01 Хр
202341831114R03C01 Хр
202341831114R04C01 Хр
202290551140R01C01 Хр
202341831114R05C01 Хр
202290551140R03C01 Хр
202290551140R05C01 Хр
202341831114R07C01 Хр
202290551164R01C01 Хр


In [116]:
cursor = connection.cursor()

query_join = '''
                SELECT * FROM dna_snps
                JOIN dna_char USING(dna_id)'''

cursor.execute(query_join)

In [117]:
result = cursor.fetchall()

In [118]:
for row in result[:10]:
    print(*row)

1 202341831114R01C01 [T/C] 1_10573221 10573221 0 839 Д Хр
2 202341831114R01C01 [T/C] 1_10673082 10673082 0.8272 839 Д Хр
3 202341831114R01C01 [A/G] 1_10723065 10723065 0.8316 839 Д Хр
4 202341831114R01C01 [A/G] 1_11337555 11337555 0.3781 839 Д Хр
5 202341831114R01C01 [A/G] 1_11407894 11407894 0.9038 839 Д Хр
6 202341831114R01C01 [T/C] 1_11426075 11426075 0.9043 839 Д Хр
7 202341831114R01C01 [T/C] 1_11445382 11445382 0.2429 839 Д Хр
8 202341831114R01C01 [A/G] 1_135128255 135128255 0 839 Д Хр
9 202341831114R01C01 [T/C] 1_13996200 13996200 0.8237 839 Д Хр
10 202341831114R01C01 [A/G] 1_142535524 142535524 0.9271 839 Д Хр
